In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
import re 
from scipy import sparse
import time
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=300
pd.options.display.max_columns = 100

from sklearn.linear_model import Ridge


In [ ]:
!pip install emoji

     |████████████████████████████████| 170 kB 5.4 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=1a0db2fcad8aa23edd088faaa9b44ca01d265675aeb3095b225235f79d6825fd
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [108]:
import re
import sys
import unicodedata

from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer

import emoji

CUSTOM_TABLE = str.maketrans(
    {
        "\xad": None,
        "\x7f": None,
        "\ufeff": None,
        "\u200b": None,
        "\u200e": None,
        "\u202a": None,
        "\u202c": None,
        "‘": "'",
        "’": "'",
        "`": "'",
        "“": '"',
        "”": '"',
        "«": '"',
        "»": '"',
        "ɢ": "G",
        "ɪ": "I",
        "ɴ": "N",
        "ʀ": "R",
        "ʏ": "Y",
        "ʙ": "B",
        "ʜ": "H",
        "ʟ": "L",
        "ғ": "F",
        "ᴀ": "A",
        "ᴄ": "C",
        "ᴅ": "D",
        "ᴇ": "E",
        "ᴊ": "J",
        "ᴋ": "K",
        "ᴍ": "M",
        "Μ": "M",
        "ᴏ": "O",
        "ᴘ": "P",
        "ᴛ": "T",
        "ᴜ": "U",
        "ᴡ": "W",
        "ᴠ": "V",
        "ĸ": "K",
        "в": "B",
        "м": "M",
        "н": "H",
        "т": "T",
        "ѕ": "S",
        "—": "-",
        "–": "-",
    }
)

WORDS_REPLACER = [
    ("sh*t", "shit"),
    ("s**t", "shit"),
    ("f*ck", "fuck"),
    ("fu*k", "fuck"),
    ("f**k", "fuck"),
    ("f*****g", "fucking"),
    ("f***ing", "fucking"),
    ("f**king", "fucking"),
    ("p*ssy", "pussy"),
    ("p***y", "pussy"),
    ("pu**y", "pussy"),
    ("p*ss", "piss"),
    ("b*tch", "bitch"),
    ("bit*h", "bitch"),
    ("h*ll", "hell"),
    ("h**l", "hell"),
    ("cr*p", "crap"),
    ("d*mn", "damn"),
    ("stu*pid", "stupid"),
    ("st*pid", "stupid"),
    ("n*gger", "nigger"),
    ("n***ga", "nigger"),
    ("f*ggot", "faggot"),
    ("scr*w", "screw"),
    ("pr*ck", "prick"),
    ("g*d", "god"),
    ("s*x", "sex"),
    ("a*s", "ass"),
    ("a**hole", "asshole"),
    ("a***ole", "asshole"),
    ("a**", "ass"),
]

REGEX_REPLACER = [
    (re.compile(pat.replace("*", "\*"), flags=re.IGNORECASE), repl)
    for pat, repl in WORDS_REPLACER
]

RE_SPACE = re.compile(r"\s")
RE_MULTI_SPACE = re.compile(r"\s+")

NMS_TABLE = dict.fromkeys(
    i for i in range(sys.maxunicode + 1) if unicodedata.category(chr(i)) == "Mn"
)

HEBREW_TABLE = {i: "א" for i in range(0x0590, 0x05FF)}
ARABIC_TABLE = {i: "ا" for i in range(0x0600, 0x06FF)}
CHINESE_TABLE = {i: "是" for i in range(0x4E00, 0x9FFF)}
KANJI_TABLE = {i: "ッ" for i in range(0x2E80, 0x2FD5)}
HIRAGANA_TABLE = {i: "ッ" for i in range(0x3041, 0x3096)}
KATAKANA_TABLE = {i: "ッ" for i in range(0x30A0, 0x30FF)}

TABLE = dict()
TABLE.update(CUSTOM_TABLE)
TABLE.update(NMS_TABLE)
# Non-english languages
TABLE.update(CHINESE_TABLE)
TABLE.update(HEBREW_TABLE)
TABLE.update(ARABIC_TABLE)
TABLE.update(HIRAGANA_TABLE)
TABLE.update(KATAKANA_TABLE)
TABLE.update(KANJI_TABLE)


EMOJI_REGEXP = emoji.get_emoji_regexp()

UNICODE_EMOJI_MY = {
    k: f" EMJ {v.strip(':').replace('_', ' ')} "
    for k, v in emoji.UNICODE_EMOJI_ALIAS_ENGLISH.items()
}


def my_demojize(string: str) -> str:
    """
    Modified version of the demojize function from emoji
    """

    def replace(match):
        return UNICODE_EMOJI_MY.get(match.group(0), match.group(0))

    return re.sub("\ufe0f", "", EMOJI_REGEXP.sub(replace, string))


def fix_tokens(tokens):
    """
    Expects a list of lower-cased tokens from TweeterTokenizer
    """
    for token in tokens:
        if token == "gov't" or token == "govt":
            yield "government"
        elif token == "i'm":
            yield "i"
            yield "am"
        elif token.endswith("n't"):
            yield token[:-3]
            yield "not"
        elif token.endswith("'re"):
            yield token[:-3]
            yield "are"
        elif token.endswith("'ll"):
            yield token[:-3]
            yield "will"
        elif token.endswith("'ve"):
            yield token[:-3]
            yield "have"
        elif token.endswith("'s"):
            yield token[:-2]
            yield "'s"
        else:
            yield token


def normalize(text: str) -> str:
    text = my_demojize(text)
    text = RE_SPACE.sub(" ", text)
    text = unicodedata.normalize("NFKD", text)
    text = text.translate(TABLE)
    text = RE_MULTI_SPACE.sub(" ", text).strip()

    for pattern, repl in REGEX_REPLACER:
        text = pattern.sub(repl, text)

    return text

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/Jigsaw/FastText/oversampled.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,text,y
0,0,49214,oh why dear johnny boy part one again do you want a fucking fight what the shit is your problem ooh he s personally attacking me grow up,2.08
1,1,91718,savant13 is a fat shite,3.58
2,2,83243,i am english i am i do not want to get blocked again fu k,0.80
3,3,112510,and why the fuck should i care g e t r i d o f a l l i d i d p l e a s e j a ck a s s,4.58
4,4,92430,yo dear sir james paul you a fruity ass nigga get a life man unless your happy with editing wikipedia as your profession feel bad for you its probably hard livin life without any friends,2.08


In [128]:
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string


    # text = text.lower()
    # Clean some punctutations
    text = text.replace('\n', ' \n ')
    # Remove all numbers
    text = re.sub(r'[0-9]', '', text)
    # Add space between pattern
    text = re.sub(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3', text)
    # Replace repeating puncts more than 3 times to length of 3
    text = re.sub(r'([*!?\'])\1\1{2,}',r'\1\1\1', text)    
    # Add space around repeating puncts
    text = re.sub(r'([*!?\']+)',r' \1 ', text)    
    # patterns with repeating characters 
    # naaaahhhhhhh to naaaahh
    text = re.sub(r'([a-zA-Z])\1{2,}\b',r'\1\1', text)
    # naaaahh to naaahh
    text = re.sub(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1', text)
    # multiple spaces to one
    text = re.sub(r'[ ]{2,}',' ', text).strip()   
    text = normalize(text)


    return text

In [ ]:
df = df.dropna(axis = 0)

In [ ]:
vec = TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (3,5), max_features = 46000)
vec.fit(df['text'])

TfidfVectorizer(analyzer='char_wb', max_df=0.5, max_features=46000, min_df=3,
                ngram_range=(3, 5))

In [ ]:
# pip install gensim=="4.0.1"

In [ ]:
from gensim.models import KeyedVectors, FastText

fmodel = FastText.load('/content/drive/Shareddrives/Jigsaw/FastText/Jigsaw-Fasttext-Word-Embeddings-256D/Jigsaw-Fasttext-Word-Embeddings-256D.bin')

In [ ]:
from scipy.sparse import hstack

def splitter(text):
    tokens = []
    
    for word in text.split(' '):
        tokens.append(word)
    
    return tokens

def vectorizer(text):
    tokens = splitter(text)
    
    x1 = vec.transform([text]).toarray()
    x2 = np.mean(fmodel.wv[tokens], axis = 0).reshape(1, -1)
    x = np.concatenate([x1, x2], axis = -1).astype(np.float16)
    del x1
    del x2 
    
    return x   

In [ ]:
X_list = []

for text in df.text:
    X_list.append(vectorizer(text))

In [ ]:
EMB_DIM = len(vec.vocabulary_) + 256

In [ ]:
X_np = np.array(X_list).reshape(-1, EMB_DIM)

In [ ]:
from scipy import sparse

X = sparse.csr_matrix(X_np)
del X_np

In [ ]:
# %%time
# model = Ridge(alpha=0.5)
# model.fit(X, df['y'])

CPU times: user 5min 25s, sys: 9min 3s, total: 14min 29s
Wall time: 3min 43s


In [ ]:
# %%time
# l_model = Ridge(alpha=1.)
# l_model.fit(X, df['y'])

CPU times: user 4min 13s, sys: 6min 52s, total: 11min 6s
Wall time: 2min 54s


In [ ]:
# %%time
# d_model = Ridge(alpha=1.5)
# d_model.fit(X, df['y'])

CPU times: user 4min 7s, sys: 6min 54s, total: 11min 1s
Wall time: 2min 49s


In [ ]:
# %%time
# s_model = Ridge(alpha=2.)
# s_model.fit(X, df['y'])

CPU times: user 3min 43s, sys: 6min 11s, total: 9min 55s
Wall time: 2min 33s


In [ ]:
# import pickle

# with open('model.pkl','wb') as f:
#     pickle.dump(model,f)
# with open('lmodel.pkl','wb') as f:
#     pickle.dump(l_model,f)
# with open('dmodel.pkl','wb') as f:
#     pickle.dump(d_model,f)
# with open('smodel.pkl','wb') as f:
#     pickle.dump(s_model,f)

In [ ]:
# import pickle

# with open('../input/fasttext-ridge/model.pkl','rb') as f:
#     model = pickle.load(f)
# with open('../input/fasttext-ridge/lmodel.pkl','rb') as f:
#     l_model = pickle.load(f)
# with open('../input/fasttext-ridge/dmodel.pkl','rb') as f:
#     d_model = pickle.load(f)
# with open('../input/fasttext-ridge/smodel.pkl','rb') as f:
#     s_model = pickle.load(f)

In [110]:
df_val = pd.read_csv("/content/drive/Shareddrives/Jigsaw/validation_data.csv")

In [ ]:
# df_val = pd.read_csv("/content/drive/Shareddrives/Jigsaw/validation_true.csv")

In [129]:
tqdm.pandas()
df_val['less_toxic'] = df_val['less_toxic'].progress_apply(text_cleaning)
df_val['more_toxic'] = df_val['more_toxic'].progress_apply(text_cleaning)

  0%|          | 0/30108 [00:00<?, ?it/s]

  0%|          | 0/30108 [00:00<?, ?it/s]

In [130]:
df_val.to_csv("cleaned_valid4.csv")

In [131]:
X_less_toxic_temp = []

for text in df_val.less_toxic:
    X_less_toxic_temp.append(vectorizer(text))

X_less_toxic_temp = np.array(X_less_toxic_temp).reshape(-1, EMB_DIM)
X_less_toxic = sparse.csr_matrix(X_less_toxic_temp)

del X_less_toxic_temp
    
X_more_toxic_temp = []

for text in df_val.more_toxic:
    X_more_toxic_temp.append(vectorizer(text))
    
X_more_toxic_temp = np.array(X_more_toxic_temp).reshape(-1, EMB_DIM)
X_more_toxic = sparse.csr_matrix(X_more_toxic_temp)

del X_more_toxic_temp

In [132]:
less1 = model.predict(X_less_toxic)
more1 = model.predict(X_more_toxic)
# Validation Accuracy
(less1 < more1).mean()

0.6719808688720605

In [133]:
less2  = l_model.predict(X_less_toxic)
more2 = l_model.predict(X_more_toxic)
# Validation Accuracy
(less2 < more2).mean()

0.6752358177228643

In [134]:
less3 = d_model.predict(X_less_toxic)
more3 = d_model.predict(X_more_toxic)
# Validation Accuracy
(less3 < more3).mean()

0.6769961472034011

In [135]:
less4 = s_model.predict(X_less_toxic)
more4 = s_model.predict(X_more_toxic)
# Validation Accuracy
(less4 < more4).mean()

0.6777600637704264

In [ ]:
# score_less =(less1 + less2 + less3 + less4) / 4
# score_more =(more1 + more2 + more3 + more4) / 4
# score_df = pd.DataFrame()
# score_df["less_score"] = score_less
# score_df["more_score"] = score_more
# score_df.to_csv("/content/drive/Shareddrives/Jigsaw/Blending/fasttext.csv")

# Prepare submission data 

In [ ]:
df_sub = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")

<h2>Text cleaning</h2>

In [ ]:
tqdm.pandas()
df_sub['text'] = df_sub['text'].progress_apply(text_cleaning)

Preparing Submission data

In [ ]:
X_sub_temp = []
for text in df_sub.text:
    X_sub_temp.append(vectorizer(text))
    
X_sub_temp = np.array(X_sub_temp).reshape(-1, EMB_DIM)
X_test = sparse.csr_matrix(X_sub_temp)

del X_sub_temp

In [ ]:
p3 = model.predict(X_test)

In [ ]:
p4 = l_model.predict(X_test)
p5 = s_model.predict(X_test)
p6 = d_model.predict(X_test)

In [ ]:
df_sub['score'] = (p3 + p4 + p5 + p6) / 4.

In [ ]:
df_sub['score'].count()

In [ ]:
df_sub['score'] = df_sub['score'] 

In [ ]:
df_sub['score'].nunique()

<h2>Prepare submission file</h2>

In [ ]:
df_sub[['comment_id', 'score']].to_csv("submission.csv", index=False)